In [1]:
import os
import time
import torch
import cache_dit
import subprocess
from diffusers import ZImagePipeline, AutoModel, PyramidAttentionBroadcastConfig
from diffusers import BitsAndBytesConfig as DiffusersBitsAndBytesConfig
from transformers import BitsAndBytesConfig as TransformersBitsAndBytesConfig

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

def load_model(model_name= 'Tongyi-MAI/Z-Image-Turbo', cache_dir='/root/autodl-tmp/Model'):
    device = "cuda"
    quantization_config = DiffusersBitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        llm_int8_skip_modules=["transformer_blocks.0.img_mod"],
    )
    transformer = AutoModel.from_pretrained(
        model_name,
        cache_dir=cache_dir,
        subfolder="transformer",
        quantization_config=quantization_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        mirror='https://hf-mirror.com'
    )
    transformer.set_attention_backend('flash')
    transformer.to(memory_format=torch.channels_last)
    # transformer = transformer.to("cpu")

    quantization_config = TransformersBitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )
    text_encoder = AutoModel.from_pretrained(
        model_name,
        cache_dir=cache_dir,
        subfolder="text_encoder",
        quantization_config=quantization_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        mirror='https://hf-mirror.com'
    )
    # text_encoder = text_encoder.to("cpu")

    pipe = ZImagePipeline.from_pretrained(
        model_name,
        transformer=transformer,
        text_encoder=text_encoder,
        torch_dtype=torch.bfloat16,
        mirror='https://hf-mirror.com',
        device_map=device
    )
    # pipe.enable_model_cpu_offload(gpu_id=gpu_id)
    return pipe

def image_generate(pipeline, prompt, special_name, num_inference_steps, seed=10086):
    s_time = time.time()
    image = pipeline(
        prompt=prompt,
        height=1024,
        width=1024,
        num_inference_steps=num_inference_steps,
        guidance_scale=0.0,
        generator=torch.Generator("cuda").manual_seed(seed),
    ).images[0]
    e_time = time.time()
    if special_name:
        image.save(f"./tmp/{special_name}-{e_time- s_time:.2f}.png")
    print(f"Used Time: {e_time- s_time:.2f}")
prompt = "超写实亚洲中年男性，年龄约45-55岁。面容坚毅、憔悴，带有生活阅历的痕迹（如眼角的细纹）。他穿着质感柔软的深灰色高领毛衣，外搭一件经典的卡其色风衣，站在寒风中周围是高楼大厦"
z_image_pipeline = load_model()

/root/autodl-tmp/miniconda3/envs/z-image/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.22s/it]
Attention backends are an experimental feature and the API may be subject to change.
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.05it/s]
Keyword arguments {'mirror': 'https://hf-mirror.com'} are not expected by ZImagePipeline and will be ignored.
Loading pipeline components...: 100%|██████████| 5/5 [00:00<00:00, 19.49it/s]


# torch.compile加速

In [ ]:
z_image_pipeline = load_model()
z_image_pipeline.transformer.compile()
s_time = time.time()
for i in range(5):
    image_generate(z_image_pipeline,
                   "Realistic mid-aged male image",
                   None)
print(f"Used Time: {time.time()- s_time:.2f}")

In [ ]:
z_image_pipeline = load_model()
s_time = time.time()
for i in range(10):
    image_generate(z_image_pipeline,
                   "Realistic mid-aged male image",
                   None)
print(f"Used Time: {time.time()- s_time:.2f}")

# CacheDit使用  
对于量化、attention后端选择等可以直接看文档：[https://cache-dit.readthedocs.io/en/latest/user_guide/ATTENTION/#available-backend](https://cache-dit.readthedocs.io/en/latest/user_guide/ATTENTION/#available-backend)
## 基础使用方式
```python
cache_dit.enable_cache(dit_pipeline)
cache_dit.disable_cache(dit_pipeline)
```
直接运行上面代码就可以启动cache_dit，除此之外可以直接使用：
```python
stats = cache_dit.summary(pipe)
....
🤗Context Options: ZImageTransformerBlock

{'cache_config': DBCacheConfig(cache_type=<CacheType.DBCache: 'DBCache'>, Fn_compute_blocks=8, Bn_compute_blocks=8, residual_diff_threshold=0.12, max_accumulated_residual_diff_threshold=None, max_warmup_steps=2, warmup_interval=1, max_cached_steps=-1, max_continuous_cached_steps=-1, enable_separate_cfg=False, cfg_compute_first=False, cfg_diff_compute_separate=True, num_inference_steps=None, steps_computation_mask=None, steps_computation_policy='dynamic'), 'name': 'layers_139918048080288'}
WARNING 01-14 22:27:20 [summary.py:288] Can't find Parallelism Config for: ZImageTransformerBlock

⚡️Cache Steps and Residual Diffs Statistics: ZImageTransformerBlock

| Cache Steps | Diffs P00 | Diffs P25 | Diffs P50 | Diffs P75 | Diffs P95 | Diffs Min | Diffs Max |
|-------------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|
| 0           | 0.205     | 0.242     | 0.273     | 0.377     | 0.711     | 0.205     | 0.852     |

WARNING 01-14 22:27:20 [summary.py:276] Can't find Context Options for: ZImageTransformerBlock
WARNING 01-14 22:27:20 [summary.py:288] Can't find Parallelism Config for: ZImageTransformerBlock

🤗Context Options: ZImageTransformer2DModel

{'cache_config': DBCacheConfig(cache_type=<CacheType.DBCache: 'DBCache'>, Fn_compute_blocks=8, Bn_compute_blocks=8, residual_diff_threshold=0.12, max_accumulated_residual_diff_threshold=None, max_warmup_steps=2, warmup_interval=1, max_cached_steps=-1, max_continuous_cached_steps=-1, enable_separate_cfg=False, cfg_compute_first=False, cfg_diff_compute_separate=True, num_inference_steps=None, steps_computation_mask=None, steps_computation_policy='dynamic'), 'name': 'layers_139918048080288'}
WARNING 01-14 22:27:20 [summary.py:288] Can't find Parallelism Config for: ZImageTransformer2DModel

⚡️Cache Steps and Residual Diffs Statistics: ZImageTransformer2DModel

| Cache Steps | Diffs P00 | Diffs P25 | Diffs P50 | Diffs P75 | Diffs P95 | Diffs Min | Diffs Max |
|-------------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|
| 0           | 0.205     | 0.242     | 0.273     | 0.377     | 0.711     | 0.205     | 0.852     |
....

```
去分析特征之间的残差分布情况（必须先进行推理之后才能进行）而后去确定参数`residual_diff_threshold`比如说：
```python
fn, bn = 8, 8
warmup_ratio, threshold = 0.2, 0.12
num_inference_steps = 10
cache_dit.enable_cache(
    z_image_pipeline,
    cache_config=DBCacheConfig(
        max_warmup_steps=int(num_inference_steps* warmup_ratio), # 多少步不去cache缓存
        max_cached_steps=-1,
        Fn_compute_blocks= fn,                                   # fn层计算结果并且判断是否缓存
        Bn_compute_blocks= bn,                                   # bn层直接计算
        residual_diff_threshold= threshold,                      # 差异阈值
    ),
)
```
## BlockAdapter使用方式  
这个参数主要是为了结局在自定义的模型中使用了自定义的模型层，而这个模型层不在cache-dit中存在，比如说假设z-image里面模型有新定义的层/修改了他某些层。
```python
...
(noise_refiner): ModuleList(
    (0-1): 2 x ZImageTransformerBlock(
      (attention): Attention(
        (norm_q): RMSNorm()
        (norm_k): RMSNorm()
        (to_q): Linear4bit(in_features=3840, out_features=3840, bias=False)
        (to_k): Linear4bit(in_features=3840, out_features=3840, bias=False)
        (to_v): Linear4bit(in_features=3840, out_features=3840, bias=False)
        (to_out): ModuleList(
          (0): Linear4bit(in_features=3840, out_features=3840, bias=False)
          (1): Dropout(p=0.0, inplace=False)
        )
      )
...
```
比如说`all_final_layer`就是修改的，那么可以直接这么使用BlockAdapter 可以帮助您快速将各种缓存加速功能应用于自定义的Diffusion Pipelines和Transformers。那么可以直接使用
```python
from cache_dit import ForwardPattern, BlockAdapter
cache_dit.enable_cache(
    BlockAdapter(
        pipe= z_image_pipeline,
        transformer=z_image_pipeline.transformer,
        blocks=[
            z_image_pipeline.transformer.noise_refiner,
        ],
    ),
)
```

In [2]:
print(z_image_pipeline.transformer)

Loading checkpoint shards: 100%|██████████| 3/3 [00:08<00:00,  2.90s/it]
`torch_dtype` is deprecated! Use `dtype` instead!
Fetching 8 files: 100%|██████████| 8/8 [00:14<00:00,  1.84s/it]
Keyword arguments {'mirror': 'https://hf-mirror.com'} are not expected by ZImagePipeline and will be ignored.
Loading pipeline components...: 100%|██████████| 5/5 [00:00<00:00, 17.71it/s]

ZImageTransformer2DModel(
  (all_x_embedder): ModuleDict(
    (2-1): Linear4bit(in_features=64, out_features=3840, bias=True)
  )
  (all_final_layer): ModuleDict(
    (2-1): FinalLayer(
      (norm_final): LayerNorm((3840,), eps=1e-06, elementwise_affine=False)
      (linear): Linear4bit(in_features=3840, out_features=64, bias=True)
      (adaLN_modulation): Sequential(
        (0): SiLU()
        (1): Linear4bit(in_features=256, out_features=3840, bias=True)
      )
    )
  )
  (noise_refiner): ModuleList(
    (0-1): 2 x ZImageTransformerBlock(
      (attention): Attention(
        (norm_q): RMSNorm()
        (norm_k): RMSNorm()
        (to_q): Linear4bit(in_features=3840, out_features=3840, bias=False)
        (to_k): Linear4bit(in_features=3840, out_features=3840, bias=False)
        (to_v): Linear4bit(in_features=3840, out_features=3840, bias=False)
        (to_out): ModuleList(
          (0): Linear4bit(in_features=3840, out_features=3840, bias=False)
          (1): Dropout(p=0.0,

In [ ]:
# BlockAdapter使用方式
from cache_dit import ForwardPattern, BlockAdapter
cache_dit.enable_cache(
    BlockAdapter(
        pipe= z_image_pipeline,
        transformer=z_image_pipeline.transformer,
        blocks=[
            z_image_pipeline.transformer.noise_refiner,
        ],
    ),
)
image_generate(z_image_pipeline,
               "Realistic mid-aged male image",
               "BlockAdapter")

In [5]:
# Fn Bn使用
from cache_dit import DBCacheConfig

fn, bn = 8, 8
warmup_ratio, threshold = 0.2, 0.12
num_inference_steps = 10
cache_dit.enable_cache(
    z_image_pipeline,
    cache_config=DBCacheConfig(
        max_warmup_steps=int(num_inference_steps* warmup_ratio),
        max_cached_steps=-1,
        Fn_compute_blocks= fn,
        Bn_compute_blocks= bn, 
        residual_diff_threshold= threshold,
    ),
)
image_generate(z_image_pipeline, 
               prompt,
               f"{fn}-{bn}-{len(prompt)}-{threshold}", 
               num_inference_steps)
stats = cache_dit.summary(z_image_pipeline)
print(stats)

INFO 01-14 22:27:13 [cache_adapter.py:57] ZImagePipeline is officially supported by cache-dit. Use it's pre-defined BlockAdapter directly!
INFO 01-14 22:27:13 [block_adapters.py:220] Auto fill blocks_name: layers.
WARNING 01-14 22:27:13 [block_adapters.py:479] Skipped Forward Pattern Check: ForwardPattern.Pattern_3
INFO 01-14 22:27:13 [cache_adapter.py:150] Use default 'enable_separate_cfg' from block adapter register: False, Pipeline: ZImagePipeline.
INFO 01-14 22:27:13 [cache_adapter.py:341] Collected Context Config: DBCache_F8B8_W2I1M0MC0_R0.12_CFG0, Calibrator Config: None
WARNING 01-14 22:27:13 [pattern_base.py:78] Skipped Forward Pattern Check: ForwardPattern.Pattern_3
INFO 01-14 22:27:13 [pattern_base.py:70] Match Blocks: CachedBlocks_Pattern_3_4_5, for layers, cache_context: layers_139918048080288, context_manager: ZImagePipeline_139918049240816.


100%|██████████| 10/10 [00:05<00:00,  1.82it/s]


Used Time: 5.96
WARNING 01-14 22:27:20 [summary.py:276] Can't find Context Options for: ZImageTransformerBlock
WARNING 01-14 22:27:20 [summary.py:288] Can't find Parallelism Config for: ZImageTransformerBlock

🤗Context Options: ZImageTransformerBlock

{'cache_config': DBCacheConfig(cache_type=<CacheType.DBCache: 'DBCache'>, Fn_compute_blocks=8, Bn_compute_blocks=8, residual_diff_threshold=0.12, max_accumulated_residual_diff_threshold=None, max_warmup_steps=2, warmup_interval=1, max_cached_steps=-1, max_continuous_cached_steps=-1, enable_separate_cfg=False, cfg_compute_first=False, cfg_diff_compute_separate=True, num_inference_steps=None, steps_computation_mask=None, steps_computation_policy='dynamic'), 'name': 'layers_139918048080288'}
WARNING 01-14 22:27:20 [summary.py:288] Can't find Parallelism Config for: ZImageTransformerBlock

⚡️Cache Steps and Residual Diffs Statistics: ZImageTransformerBlock

| Cache Steps | Diffs P00 | Diffs P25 | Diffs P50 | Diffs P75 | Diffs P95 | Diffs Min 

In [2]:
# 不使用cachedit
image_generate(z_image_pipeline, 
               prompt,
               f"Normal-{len(prompt)}", 
               10)

100%|██████████| 10/10 [00:05<00:00,  1.99it/s]


Used Time: 5.67


In [ ]:
# 使用compile
z_image_pipeline.transformer = torch.compile(z_image_pipeline.transformer)
image_generate(z_image_pipeline, 
               prompt,
               f"Compile-{len(prompt)}", 
               10)

In [3]:
# 使用cachedit+ complie
cache_dit.enable_cache(z_image_pipeline)
image_generate(z_image_pipeline, 
               prompt,
               f"CacheDit-{len(prompt)}", 
               10)

INFO 01-14 22:51:00 [cache_interface.py:200] cache_config is None, using default DBCacheConfig
INFO 01-14 22:51:00 [cache_adapter.py:57] ZImagePipeline is officially supported by cache-dit. Use it's pre-defined BlockAdapter directly!
INFO 01-14 22:51:00 [block_adapters.py:220] Auto fill blocks_name: layers.
WARNING 01-14 22:51:00 [block_adapters.py:479] Skipped Forward Pattern Check: ForwardPattern.Pattern_3
INFO 01-14 22:51:00 [cache_adapter.py:150] Use default 'enable_separate_cfg' from block adapter register: False, Pipeline: ZImagePipeline.
INFO 01-14 22:51:00 [cache_adapter.py:341] Collected Context Config: DBCache_F8B0_W8I1M0MC0_R0.08_CFG0, Calibrator Config: None
WARNING 01-14 22:51:00 [pattern_base.py:78] Skipped Forward Pattern Check: ForwardPattern.Pattern_3
INFO 01-14 22:51:00 [pattern_base.py:70] Match Blocks: CachedBlocks_Pattern_3_4_5, for layers, cache_context: layers_140602253406288, context_manager: ZImagePipeline_140602228024352.


100%|██████████| 10/10 [00:05<00:00,  1.98it/s]


Used Time: 5.48


# 各类加速trick综合测试

In [ ]:
import time
import torch
import cache_dit
from diffusers import ZImagePipeline, AutoModel, PyramidAttentionBroadcastConfig
from diffusers import BitsAndBytesConfig as DiffusersBitsAndBytesConfig
from transformers import BitsAndBytesConfig as TransformersBitsAndBytesConfig

def load_model(model_name= 'Tongyi-MAI/Z-Image-Turbo', cache_dir='/root/autodl-tmp/Model'):
    device = "cuda"
    quantization_config = DiffusersBitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        llm_int8_skip_modules=["transformer_blocks.0.img_mod"],
    )
    transformer = AutoModel.from_pretrained(
        model_name,
        cache_dir=cache_dir,
        subfolder="transformer",
        quantization_config=quantization_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        mirror='https://hf-mirror.com'
    )
    # transformer = transformer.to("cpu")

    quantization_config = TransformersBitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )
    text_encoder = AutoModel.from_pretrained(
        model_name,
        cache_dir=cache_dir,
        subfolder="text_encoder",
        quantization_config=quantization_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        mirror='https://hf-mirror.com'
    )
    # text_encoder = text_encoder.to("cpu")

    pipe = ZImagePipeline.from_pretrained(
        model_name,
        transformer=transformer,
        text_encoder=text_encoder,
        torch_dtype=torch.bfloat16,
        mirror='https://hf-mirror.com',
        device_map=device
    )
    # pipe.enable_model_cpu_offload(gpu_id=gpu_id)
    return pipe

def image_generate(pipeline, prompt, special_name, seed=10086):
    s_time = time.time()
    image = pipeline(
        prompt=prompt,
        height=1024,
        width=1024,
        num_inference_steps=10,
        guidance_scale=0.0,
        generator=torch.Generator("cuda").manual_seed(seed),
    ).images[0]
    e_time = time.time()
    image.save(f"./{special_name}-{e_time- s_time:.2f}.png")

z_image_pipeline = load_model()

# 量化操作
本机参数如下：`Linux version 5.4.0-153-generic (buildd@bos03-amd64-008) (gcc version 9.4.0 (Ubuntu 9.4.0-1ubuntu1~20.04.1)) 4090 CUDA Version: 12.6,`
## 量化工具——llama.cpp安装
> 官方项目：[https://github.com/ggml-org/llama.cpp](https://github.com/ggml-org/llama.cpp)

参考[项目](https://qwen.readthedocs.io/zh-cn/latest/run_locally/llama.cpp.html#getting-the-program)中的安装方式（本地编译方式），首先检查C++环境：`cc --version`或者直接使用 `cmake --version`。llama.cpp安装开始：
```bash
git clone https://github.com/ggml-org/llama.cpp
cd llama.cpp
cmake -B build # 将检查本地环境并确定需要包含的推理后端与特性
cmake --build build --config Release -j 8 # 8个并行命令进行编译
```

https://grok.com/share/c2hhcmQtMi1jb3B5_31abfa5d-8124-4595-8745-5c52c63c77f1
## 大模型的量化操作
[量化大语言模型](https://github.com/shangxiaaabb/ProjectCode/blob/main/code/Python/DFModelCode/DF_acceralate/quant_LLM.ipynb)

https://qwen.readthedocs.io/zh-cn/latest/quantization/llama.cpp.html
## 多模态模型的量化操作
[量化多模态模型](https://github.com/shangxiaaabb/ProjectCode/blob/main/code/Python/DFModelCode/DF_acceralate/quant_MultiModal.ipynb)

## 扩散模型量化操作
[量化扩散模型](https://github.com/shangxiaaabb/ProjectCode/blob/main/code/Python/DFModelCode/DF_acceralate/quant_DFModel.ipynb)
